In [51]:
import sys
sys.path.append('../..')

In [52]:
from Data.data75 import getData

import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr

from sklearn.linear_model import LinearRegression, RidgeCV, MultiTaskLassoCV, MultiTaskElasticNetCV
from tqdm import tqdm

In [53]:
path = '../../Data'
list_name = 'stock_list.csv'
start_date = '20140106'
end_date = '20220527'

In [54]:
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list = stock_list.sort_values(by=['종목코드'])
stock_list

,종목명,종목코드,상장시장
109,메리츠화재,000060,KOSPI
126,하이트진로,000080,KOSPI
67,유한양행,000100,KOSPI
69,CJ대한통운,000120,KOSPI
156,두산,000150,KOSPI
...,...,...,...
328,넥스틴,348210,KOSDAQ
31,하이브,352820,KOSPI
199,솔브레인,357780,KOSDAQ
176,티와이홀딩스,363280,KOSPI


In [55]:
sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path, sample_name))
answer = sample_submission.copy()
answer

,Index,2022-05-23,2022-05-24,2022-05-25,2022-05-26,2022-05-27
0,60,0,0,0,0,0
1,80,0,0,0,0,0
2,100,0,0,0,0,0
3,120,0,0,0,0,0
4,150,0,0,0,0,0
...,...,...,...,...,...,...
365,348210,0,0,0,0,0
366,352820,0,0,0,0,0
367,357780,0,0,0,0,0
368,363280,0,0,0,0,0


In [56]:
common_data = getData(start_date, end_date)
common_data

,Date,weekday,weeknum,kospi_Close,kospi_Volume,kospi_Change,kosdaq_Close,kosdaq_Volume,kosdaq_Change,nasdaq_Close,jpy_Close,jpy_Change,acf_Close,acf_Change,btc_Close,btc_Volume,btc_Change
0,2014-01-06,0,02,1953.28,193630000.0,0.0037,500.62,2.887100e+05,0.0026,4113.68,10.2232,0.0156,0.8108,0.0010,3206000.0,21580.0,0.0329
1,2014-01-07,1,02,1959.44,193030000.0,0.0032,504.86,3.324500e+05,0.0085,4153.18,10.2227,-0.0000,0.8115,0.0009,3206000.0,21580.0,0.0329
2,2014-01-08,2,02,1958.96,217070000.0,-0.0002,509.78,3.687500e+05,0.0097,4165.61,10.1636,-0.0058,0.8110,-0.0006,3206000.0,21580.0,0.0329
3,2014-01-09,3,02,1946.11,276400000.0,-0.0066,511.60,3.639600e+05,0.0036,4156.19,10.1402,-0.0023,0.8072,-0.0047,3206000.0,21580.0,0.0329
4,2014-01-10,4,02,1938.54,262990000.0,-0.0039,510.19,3.996100e+05,-0.0028,4174.66,10.1946,0.0054,0.8119,0.0058,3206000.0,21580.0,0.0329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,2022-05-23,0,21,2647.38,644200000.0,0.0031,883.59,1.080000e+09,0.0042,11535.27,9.8349,-0.0125,0.6862,0.0007,37336000.0,1470.0,-0.0406
2186,2022-05-24,1,21,2605.87,851240000.0,-0.0157,865.07,1.100000e+09,-0.0210,11264.45,9.9427,0.0110,0.6820,-0.0061,37739000.0,1450.0,0.0108
2187,2022-05-25,2,21,2617.22,712610000.0,0.0044,872.69,9.893000e+08,0.0088,11434.74,9.9451,0.0002,0.6820,0.0000,37863000.0,1080.0,0.0033
2188,2022-05-26,3,21,2612.45,595850000.0,-0.0018,871.43,9.474200e+08,-0.0014,11740.65,9.9082,-0.0037,0.6809,-0.0016,37595000.0,2170.0,-0.0071


In [57]:
data_dict = dict()
x_public_dict = dict()
x_dict = dict()
y_dict = dict()

for code in tqdm(stock_list['종목코드'].values):
    imported_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']]
    data = pd.merge(common_data, imported_data, how='left', left_on='Date', right_index=True)

    col_names = list(data)
    drop_name = [c for c in col_names if not(c.endswith('Change') or c == 'Close')]

    data = data.drop(columns=drop_name)

    data = data.fillna(method='ffill')
    data = data.fillna(method='bfill')

    size = len(data) // 5 - 2
    col_size = data.shape[1]

    x = np.zeros((size, col_size))
    y = np.zeros((size, 5))

    for i in range(size):
        friday = 4 + 5 * i

        x[i] = data.iloc[friday]

        closes = []

        for j in range(5):
            next = 5 * (i + 1) + j
            closes.append(data.Close.iloc[next])

        friday_close = data.Close.iloc[friday]

        y[i] = (np.array(closes) - friday_close) / friday_close

    x_public = data.iloc[-6].to_numpy().reshape((1, -1))
    
    x_dict[code] = x
    y_dict[code] = y
    x_public_dict[code] = x_public
    data_dict[code] = data

100%|██████████| 370/370 [01:17<00:00,  4.78it/s]


In [74]:
for i, code in enumerate(tqdm(stock_list['종목코드'].values)):
    model = RidgeCV()

    x = x_dict[code]
    y = y_dict[code]
    x_public = x_public_dict[code]
    data = data_dict[code]

    model.fit(x, y)

    prediction = model.predict(x_public)
    #prediction = np.zeros((1, 5))
    
    friday_close = data.Close.iloc[-6]

    sample_submission.iloc[i,1:] = (friday_close * (1 + prediction))[0]
    answer.iloc[i,1:] = data.Close.iloc[-5:].to_numpy()

100%|██████████| 370/370 [00:01<00:00, 343.71it/s]


In [75]:
answer.iloc[answer['Index'].isin([31390, 36490]), 1:] = [0,0,0,0,0]
answer

,Index,2022-05-23,2022-05-24,2022-05-25,2022-05-26,2022-05-27
0,60,39750.0,39700.0,40800.0,39800.0,39800.0
1,80,35350.0,34950.0,35900.0,36150.0,36250.0
2,100,59600.0,59100.0,59700.0,59900.0,59800.0
3,120,123000.0,125500.0,125500.0,127500.0,129000.0
4,150,80400.0,79900.0,80400.0,80100.0,80400.0
...,...,...,...,...,...,...
365,348210,68400.0,66500.0,67000.0,66300.0,67000.0
366,352820,221000.0,215500.0,219500.0,214500.0,218500.0
367,357780,269900.0,260600.0,260600.0,270900.0,271800.0
368,363280,24500.0,24400.0,24550.0,24600.0,24400.0


In [76]:
sample_submission.iloc[sample_submission['Index'].isin([31390, 36490]), 1:] = [0,0,0,0,0]
sample_submission

,Index,2022-05-23,2022-05-24,2022-05-25,2022-05-26,2022-05-27
0,60,39097.889806,39204.758056,39261.425841,39553.880492,39402.916831
1,80,35594.624029,35728.237887,35676.505622,35588.084630,35463.460454
2,100,58686.458903,58596.824733,58639.260224,58543.995279,58621.307015
3,120,124719.932350,124584.995003,124948.253888,125296.892531,125662.765211
4,150,79593.243353,79971.455057,80217.878897,80504.175435,80575.345634
...,...,...,...,...,...,...
365,348210,67923.377234,68063.138972,68323.874869,68628.884329,68772.441507
366,352820,220974.952313,221827.114115,222180.385947,222987.012831,222797.901015
367,357780,262951.809641,263292.244184,263206.901606,263395.273293,263242.922311
368,363280,24638.513552,24828.534008,24949.249540,25084.649073,25063.967348


In [77]:
result_arr = sample_submission.iloc[:,1:].to_numpy()
answer_arr = answer.iloc[:,1:].to_numpy()

print(np.mean(np.abs(result_arr - answer_arr)))

1800.209184031484


In [78]:
sample_submission.to_csv('submission.csv', index=False)